<a href="https://colab.research.google.com/github/StevenVuong/MSc_Project/blob/master/v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install dependencies
!pip install deepbrain
!pip install pydicom

     |████████████████████████████████| 686kB 25.7MB/s 
     |████████████████████████████████| 7.0MB 43.9MB/s 


In [2]:
from google.colab import drive

# mount google drive into google colab
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import os

# go to where the data is
print (os.listdir())
os.chdir('gdrive/My Drive/msc_project')
os.listdir()

['.config', 'gdrive', 'sample_data']


['t1_scan',
 'T1_SAG_SIEMEN_3T_CLEAN_5_29_2019.csv',
 'T1_SAG_SIEMEN_3T_CLEAN_1',
 'T1_SAG_SIEMEN_3T_CLEAN_5_29_2019.gsheet']

In [4]:
import pandas as pd

patients = os.listdir('T1_SAG_SIEMEN_3T_CLEAN_1') # get all patients ID's in scan
patient_df = pd.read_csv('T1_SAG_SIEMEN_3T_CLEAN_5_29_2019.csv') # get dataframe too to cross reference

patient_df.head() # so we have a dataframe of our patients' data

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,1130198,75422,GenCohort Unaff,M,73,1,MRI,MPRAGE GRAPPA,Original,11/13/2018,DCM,5/07/2019
1,1130190,75414,GenCohort Unaff,F,73,1,MRI,Sag MPRAGE GRAPPA,Original,12/13/2018,DCM,4/24/2019
2,1130191,75414,GenCohort Unaff,F,73,1,MRI,Sag MPRAGE GRAPPA,Original,12/13/2018,DCM,4/24/2019
3,1125041,74375,GenCohort Unaff,F,59,1,MRI,MPRAGE_GRAPPA,Original,9/06/2018,DCM,4/24/2019
4,1003469,72138,GenCohort Unaff,F,55,1,MRI,MPRAGE GRAPPA,Original,2/19/2018,DCM,4/24/2019


In [0]:
def get_grappa_dir(path):
  # get the file ending with 'GRAPPA', would need to accomodate this for grappa also
  for next_path in os.listdir(path):
    if (next_path.split("_")[-1] == 'GRAPPA'): # for the t1 weighted
      return next_path

In [0]:
def get_dcm_s(path):
  # get the path beginning with S, so doesn't clash with GZ File
  for next_path in os.listdir(path):
    if (next_path[0] == 'S'):
      return next_path

In [0]:
def get_img_no(path):
  # get the image identification numberm any image will do for this so take first
  image_number = None
  for image_file in os.listdir(path):
    image_number = int(image_file.split("_")[-1][1:-4]) # index to get the ID
   
  return image_number

In [0]:
def filename_sort(filename):
    
    # split by underlines and delimiter
    split_line = filename.split("_")
    int_return = int(split_line[-3])
    
    return int_return

In [10]:
import pydicom
import numpy as np

cwd = os.getcwd()
print ("Current Working Dir: %s " % cwd)

for patient in patients[:2]:
  # label = patient_df.get_value(patient, 'Subject') # cannot go by patient, must get the ID
  path = cwd + '/' + 'T1_SAG_SIEMEN_3T_CLEAN_1/' + patient # get to the GRAPPA 
  path = path + '/' + get_grappa_dir(path)
  path = path + '/' + os.listdir(path)[-1] # get the most recent scan for patient
  path = path + '/' + get_dcm_s(path)
  
  # get information related around the image
  image_number = get_img_no(path)
  image_row = patient_df.loc[patient_df['Image Data ID'] == image_number] # relate to df
  image_sex = image_row.Sex.values[0]
  image_group = image_row.Group.values[0]
  image_age = image_row.Age.values[0]
  
  print (image_sex)
  print (image_age)
  
  # get files and sort them in order
  dcm_files = os.listdir(path)
  dcm_files = sorted(dcm_files, key=lambda filename: filename_sort(filename)) # some have length 3
  
  slices = []
  # loop through slices and build the array
  for dcm_file in dcm_files:
    path_to_file = path + '/' + dcm_file
    slices.append(pydicom.read_file(path_to_file).pixel_array)
  slices = np.array(slices)
  
  print (np.shape(slices)) # each patient has different number of slices, have to work through

Current Working Dir: /content/gdrive/My Drive/msc_project 
F
50
['PPMI_3108_MR_MPRAGE_GRAPPA__br_raw_20110502103140647_1_S107265_I232161.dcm', 'PPMI_3108_MR_MPRAGE_GRAPPA__br_raw_20110502103140835_2_S107265_I232161.dcm', 'PPMI_3108_MR_MPRAGE_GRAPPA__br_raw_20110502103140803_3_S107265_I232161.dcm', 'PPMI_3108_MR_MPRAGE_GRAPPA__br_raw_20110502103140882_4_S107265_I232161.dcm', 'PPMI_3108_MR_MPRAGE_GRAPPA__br_raw_20110502103140678_5_S107265_I232161.dcm']
(176, 256, 240)
M
53
['PPMI_3179_MR_SAG_MPRAGE_GRAPPA__br_raw_20131218142906376_1_S208445_I402104.dcm', 'PPMI_3179_MR_SAG_MPRAGE_GRAPPA__br_raw_20131218142906454_2_S208445_I402104.dcm', 'PPMI_3179_MR_SAG_MPRAGE_GRAPPA__br_raw_20131218142906532_3_S208445_I402104.dcm', 'PPMI_3179_MR_SAG_MPRAGE_GRAPPA__br_raw_20131218142906610_4_S208445_I402104.dcm', 'PPMI_3179_MR_SAG_MPRAGE_GRAPPA__br_raw_20131218142905892_5_S208445_I402104.dcm']
(192, 256, 240)
